In [2]:
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import requests
session = requests.session()

In [3]:
stw = [] #cловарь стоп-слов
with open('stop-ru.txt', encoding='utf-8') as f:
  d = f.readlines()
  for i in d:
    stw.append(i.strip())

In [4]:
def tokenization(feedback): #токенизация, очищение от знаков препинания и лемматизация
  liste = []
  for i in feedback.split():
    i = i.strip(punctuation)
    if i.isalpha():
      i = morph.parse(i.lower())[0].normal_form
      if i not in stw:
        liste.append(i.lower())
  return liste

In [18]:
pos_list = [] #список слов, относящихся положительных отзывов
neg_list = [] #список слов, относящихся отрицательных отзывов

url = 'https://irecommend.ru/content/purina-one-dlya-domashnikh-koshek?ft%5Br%5D=1'
req = session.get(url).text
soup = BeautifulSoup(req)
reviews = soup.find_all('span', {'class':"reviewTeaserText"})
for i in reviews:
  pos_list.extend(tokenization(i.text))
url1 = "https://irecommend.ru/content/purina-one-dlya-domashnikh-koshek?ft%5Br%5D=0"
req1 = session.get(url1).text
soup1 = BeautifulSoup(req1)
reviews1 = soup1.find_all('span', {'class':"reviewTeaserText"})
for i in reviews1:
  neg_list.extend(tokenization(i.text))
url1_2 = "https://irecommend.ru/content/royal-canin-babycat-34-dlya-kotyat-ot-1-do-4-mesyatsev?ft%5Br%5D=0"
req2 = session.get(url1_2).text
soup2 = BeautifulSoup(req2)
reviews2 = soup2.find_all('span', {'class':"reviewTeaserText"})
for i in reviews2:
  neg_list.extend(tokenization(i.text))
url1_3 = "https://irecommend.ru/content/dlya-britantsev?ft%5Br%5D=0"
req3 = session.get(url1_3).text
soup3 = BeautifulSoup(req3)
reviews3 = soup3.find_all('span', {'class':"reviewTeaserText"})
for i in reviews3:
  neg_list.extend(tokenization(i.text))

In [14]:
freq = dict() #словарь частотности положительных слов
freq1 = dict() #словарь частотности отрицательных слов
for i in pos_list:
  freq[i] = freq.setdefault(i, 0) + 1
for i in neg_list:
  freq1[i] = freq1.setdefault(i, 0) + 1

In [24]:
set_pos = set(freq.keys())
set_neg = set(freq1.keys())
only_pos = set_pos - set_neg
only_neg = set_neg - set_pos
slov_pos = [] #только в списке положительных
slov_neg = [] #только в списке отрицательных
for i in only_pos:
  if freq[i] > 2:
    slov_pos.append(i)
for i in only_neg:
  if freq1[i] > 2:
    slov_neg.append(i)
print(slov_pos)
print(slov_neg)

['гранула', 'неплохой', 'содержание', 'злак', 'кг', 'эконом', 'попробовать', 'начаться', 'привередливый', 'капризный', 'прийти', 'бюджетный', 'выбор', 'питание', 'цельный', 'подходить', 'приятный', 'хорёк', 'приветствовать', 'дворовый']
['завести', 'вислоухий', 'месячный', 'заводчица', 'кормление', 'канин', 'беременная', 'маркет', 'начало', 'сфинкс', 'роять', 'прийтись', 'частый', 'ребёнок', 'заводчик']


In [20]:
def tone_define(feedback): #функция, определяющая тон
  tok_f = []
  for i in feedback.split():
    tok_f.extend(tokenization(i))
  k = 0
  for i in set(tok_f): #считаем число общих элементов
    for j in slov_pos:
      if i == j:
        k += 1
  coef_jac_p = k / (len(tok_f) + len(slov_pos) - k) #c помощью коэффициента \
  k = 0 #Жаккара считаем, насколько похожи множества
  for i in set(tok_f):
    for j in slov_neg:
      if i == j:
        k += 1
  coef_jac_n = k / (len(tok_f) + len(slov_neg) - k)
  if coef_jac_p > coef_jac_n:
    return 'Отзыв похож на положительный'
  elif coef_jac_p < coef_jac_n:
    return 'Отзыв похож на отрицательный'
  else:
    return 'Нельзя однозначно сказать'

In [25]:
test = []
link_tn = "https://irecommend.ru/content/korm-dlya-koshek-perfect-fit-dlya-sterilizovannykh-kotov-i-koshek?ft%5Br%5D=0"
requ = session.get(link_tn).text
soup_t = BeautifulSoup(requ)
reviews_t = soup_t.find_all('span', {'class':"reviewTeaserText"})
k = 0 #берем сначала 5 отрицательных отзывов и добавляем их в список test
for i in reviews_t:
  if k != 5:
    test.append(i.text)
    k += 1
  else:
    break
link_tp = "https://irecommend.ru/content/korm-dlya-koshek-perfect-fit-dlya-sterilizovannykh-kotov-i-koshek?ft%5Br%5D=1"
requ1 = session.get(link_tp).text
soup_tp = BeautifulSoup(requ1)
reviews_tp = soup_tp.find_all('span', {'class':"reviewTeaserText"})
p = 0 #теперь берем 5 положительных отзывов и добавляем их в test
for i in reviews_tp:
  if p != 5:
    test.append(i.text)
    p += 1
  else:
    break
for i in test:
  print(tone_define(i))

Нельзя однозначно сказать
Отзыв похож на положительный
Отзыв похож на отрицательный
Нельзя однозначно сказать
Отзыв похож на положительный
Отзыв похож на положительный
Отзыв похож на положительный
Нельзя однозначно сказать
Отзыв похож на отрицательный
Отзыв похож на отрицательный


Первые пять отзывов отрицательные, а вторые пять - положительные. Правильно определены только 3 из 10. Еще 3 невозможно определить, поскольку, возможно, в них нет слов из словаря положительных/отрицательных слов.
Для улучшения работы функции необходимы более крупные словари (нужно взять больше данных); в данном случае необходимо было бы получить доступ к полным текстам отзывов.